In [2]:
from modules.cn_allocation import Makela08_alloc_parameter
from modules.cn_allocation import Makela08_alloc_symbsolver
import numpy as np
import sympy as sp

params_dict = Makela08_alloc_parameter.get_param_dict(dict_name="params_makela08_pine")

In [6]:
# For testing purposes
Nup_max_specific = 15
Photosyn_lightsat = 10

# Wr = psi_r * Wf

psi_r_realroots_list = []
psi_r_solver = Makela08_alloc_symbsolver.PsiRCubicEqnSolver(params_dict)
for i in range(1,200):  # Nconc_foliage
    psi_r_roots, psi_r_realroots_conv, psi_r_realroots_conv_positive, coeff_list = psi_r_solver.solve_cubic_eqn_numeric(Nup_max_specific, Photosyn_lightsat, i)
    psi_r_realroots_list.append([i, psi_r_realroots_conv_positive])
    

In [7]:
# Solve G using psi_r
G_result_list = []

for indv_combination in psi_r_realroots_list:
    Nconc_foliage = indv_combination[0]
    psi_r_real = indv_combination[1][0]
    
    Wf_solver = Makela08_alloc_symbsolver.DryMassFoliageSolver(
        Nup_max_specific, Photosyn_lightsat, params_dict,
        Nconc_foliage=Nconc_foliage, use_numeric_Nconc_foliage=True
    )
    DM_foliage_C = Wf_solver.solve_carbon(psi_r_real)[0]
    DM_foliage_N = Wf_solver.solve_nitrogen(psi_r_real)[0]
    
    G_solver = Makela08_alloc_symbsolver.BiomassProductionSolver(
        params_dict,
        Nconc_foliage=Nconc_foliage, use_numeric_Nconc_foliage=True
    )
    
    G_C = G_solver.solve_total_biomass_production(DM_foliage_C, psi_r_real)
    G_N = G_solver.solve_total_biomass_production(DM_foliage_N, psi_r_real)
    
    G_result_list.append([Nconc_foliage, psi_r_real, DM_foliage_C, DM_foliage_N, G_C, G_N])
    
    